# Is it a cat? Or a plane?

In [16]:
# pass an image available via http and store it locally
import urllib

# replace this with an image of your choice :)
image_url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Equus_quagga.jpg"


urllib.urlretrieve(image_url, "/tmp/image.jpg")


('/tmp/image.jpg', <httplib.HTTPMessage instance at 0x7f02d03cc200>)

In [17]:
import numpy
import os
import sys
import tensorflow as tf
import threading

from grpc.beta import implementations
from tensorflow_serving.example import inception_inference_pb2

# server:port of the tensorflow inception-service (you shouldn't have to touch this)
server = "137.138.6.52:31435"

# some output config params (you shouldn't need to touch these either)
NUM_CLASSES = 5
WORKING_DIR = "/usr/lib/python2.7/dist-packages/tensorflow_serving/example"
SYNSET_FILE = os.path.join(WORKING_DIR, 'imagenet_lsvrc_2015_synsets.txt')
METADATA_FILE = os.path.join(WORKING_DIR, 'imagenet_metadata.txt')

# tensorflow call
host, port = server.split(":")
channel = implementations.insecure_channel(host, int(port))
stub = inception_inference_pb2.beta_create_InceptionService_stub(channel)
synsets = []
with open(SYNSET_FILE) as f:
  synsets = f.read().splitlines()

texts = {}
with open(METADATA_FILE) as f:
  for line in f.read().splitlines():
    parts = line.split('\t')
    assert len(parts) == 2
    texts[parts[0]] = parts[1]

with open("/tmp/image.jpg", 'rb') as f:
  data = f.read()
  request = inception_inference_pb2.InceptionRequest()
  request.jpeg_encoded = data  
  result = stub.Classify(request, 10.0)  # 10 secs timeout
  for i in range(NUM_CLASSES):
    index = result.classes[i]
    score = result.scores[i]
    print '%f : %s' % (score, texts[synsets[index - 1]])
